In [3]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_classes_file = "imagenet_classes.txt"
download_url(url, ".", imagenet_classes_file, None)
with open(imagenet_classes_file) as f:
    idx_to_class = [line.strip() for line in f.readlines()]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

resnet = models.resnet50(pretrained=True).eval().to(device)

transform = transforms.Compose([
    transforms.ToTensor(),
])

image_path = "cat_img.png"
img = Image.open(image_path).convert("RGB")
x = transform(img).unsqueeze(0).to(device)
x.requires_grad = True

class_prompt = "tiger cat"  
target_class_idx = idx_to_class.index(class_prompt)

output = resnet(x)
loss = output[0, target_class_idx] 

loss.backward()
epsilon = 0.01
perturbed = x - epsilon * x.grad.data.sign()
perturbed = torch.clamp(perturbed, 0, 1).detach()

from torchvision.utils import save_image
save_image(x, "original.png", normalize=True)
save_image(perturbed, "cloaked.png", normalize=True)

with torch.no_grad():
    probs_original = F.softmax(resnet(x), dim=1)[0]
    probs_cloaked = F.softmax(resnet(perturbed), dim=1)[0]

original_top = torch.topk(probs_original, 5)
cloaked_top = torch.topk(probs_cloaked, 5)

print("🔍 Top classes before cloaking:")
for i in range(5):
    print(f"{idx_to_class[original_top.indices[i]]}: {original_top.values[i].item():.4f}")

print("\n👻 Top classes after cloaking:")
for i in range(5):
    print(f"{idx_to_class[cloaked_top.indices[i]]}: {cloaked_top.values[i].item():.4f}")


/Users/hiteshgupta/Documents/mirage-ai-techjam/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hiteshgupta/Documents/mirage-ai-techjam/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: 'cat_img.png'

In [11]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from PIL import Image
import numpy as np
import random
from torchvision.utils import save_image

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
model = models.resnet50(pretrained=True).eval().to(device)

# Load image
img = Image.open("cat_img.png").convert("RGB")
transform = transforms.Compose([ transforms.ToTensor()])
x = transform(img).unsqueeze(0).to(device)

# Predict baseline
with torch.no_grad():
    base_pred = model(x).argmax(dim=1).item()

print(f"Base class index: {base_pred}")

# --- One Pixel Attack ---
def one_pixel_attack(x, model, iters=3000, n_pixels=1, epsilon=0.03):
    _, _, H, W = x.shape
    x_adv = x.clone()

    for i in range(iters):
        cand = x.clone()

        for _ in range(n_pixels):
            r = random.randint(0, H-1)
            c = random.randint(0, W-1)

            # small local perturbation, not random RGB
            delta = torch.randn(3).to(x.device) * epsilon
            cand[0, :, r, c] = torch.clamp(cand[0, :, r, c] + delta, 0, 1)

        out = model(cand)
        pred = out.argmax(dim=1).item()

        if pred != base_pred:
            print(f"Success at iter {i}, predicted {pred}")
            return cand.detach()

    return x_adv.detach()


x_adv = one_pixel_attack(x, model, iters=1000, n_pixels=10)
save_image(x_adv, "cloaked_pixel.png", normalize=False)

new_pred = model(x_adv).argmax(dim=1).item()
print(f"New class index after one-pixel attack: {new_pred}")


Base class index: 299


KeyboardInterrupt: 

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from PIL import Image
import numpy as np
import random
from torchvision.utils import save_image

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
model = models.resnet50(pretrained=True).eval().to(device)

# Load full resolution image
orig_img = Image.open("cat_img.png").convert("RGB")
W_orig, H_orig = orig_img.size

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

x_small = transform(orig_img).unsqueeze(0).to(device)
x_small.requires_grad = False  # not using FGSM/PGD now

# Predict baseline
with torch.no_grad():
    base_pred = model(x_small).argmax(dim=1).item()
print("Base class:", base_pred)

# Convert original to tensor
orig_tensor = transforms.ToTensor()(orig_img).to(device)

# ========== One-Pixel Attack (small version) ==========
def one_pixel_attack_highres(x_small, orig_tensor, model, iters=3000, n_pixels=3, epsilon=0.03):

    _, _, H_small, W_small = x_small.shape
    _, _, H_big, W_big = orig_tensor.unsqueeze(0).shape

    x_adv_small = x_small.clone()
    orig_big = orig_tensor.clone()

    for i in range(iters):
        cand_small = x_small.clone()
        cand_big = orig_big.clone()

        for _ in range(n_pixels):
            # choose attack pixel in low-res
            r_small = random.randint(0, H_small - 1)
            c_small = random.randint(0, W_small - 1)

            # small delta
            delta = torch.randn(3).to(device) * epsilon
            cand_small[0, :, r_small, c_small] = torch.clamp(
                cand_small[0, :, r_small, c_small] + delta,
                0, 1
            )

            # === Map to high-res ===
            r_big = int(r_small * H_big / H_small)
            c_big = int(c_small * W_big / W_small)

            cand_big[:, r_big, c_big] = torch.clamp(
                cand_big[:, r_big, c_big] + delta,
                0, 1
            )

        out = model(cand_small)
        pred = out.argmax(dim=1).item()

        if pred != base_pred:
            print(f"Success at iter {i}, predicted {pred}")
            return cand_big  # RETURN HIGH-RES CLOAKED VERSION

    return orig_big


x_adv_big = one_pixel_attack_highres(x_small, orig_tensor, model, iters=2000, n_pixels=5)

# Save high-resolution result
save_image(x_adv_big, "cloaked_highres.png", normalize=False)
print("Saved cloaked_highres.png")


In [15]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.utils import save_image
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# Download imagenet labels
url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_classes_file = "imagenet_classes.txt"
download_url(url, ".", imagenet_classes_file, None)
with open(imagenet_classes_file) as f:
    idx_to_class = [line.strip() for line in f.readlines()]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

resnet = models.resnet50(pretrained=True).eval().to(device)

# Load ORIGINAL high-res image
image_path = "cat_img.png"
orig_img = Image.open(image_path).convert("RGB")
orig_w, orig_h = orig_img.size

# Preprocess for model (224x224)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

x_small = preprocess(orig_img).unsqueeze(0).to(device)
x_small.requires_grad = True

# ---- Choose the target class ----
class_prompt = "tiger cat"
target_class_idx = idx_to_class.index(class_prompt)

# ---- Forward ----
output = resnet(x_small)
loss = output[0, target_class_idx]

# ---- Backward ----
loss.backward()

epsilon = 0.01
grad_sign = x_small.grad.data.sign()

# ---- Generate 224 × 224 perturbed image ----
pert_small = torch.clamp(x_small - epsilon * grad_sign, 0, 1).detach()

# ---- Map the perturbation back to original resolution ----
# Resize perturbed-small to original shape using bilinear upsampling
pert_big = torch.nn.functional.interpolate(
    pert_small, size=(orig_h, orig_w), mode='bilinear', align_corners=False
)

# But interpolation scales pixel values - we want ONLY the perturbation.
# So compute δ = pert_small - x_small, then upscale δ and add to original.

delta_small = pert_small - x_small

delta_big = torch.nn.functional.interpolate(
    delta_small, size=(orig_h, orig_w), mode='bilinear', align_corners=False
)

# Convert original high-res to tensor
orig_tensor = transforms.ToTensor()(orig_img).unsqueeze(0).to(device)

perturbed_highres = torch.clamp(orig_tensor + delta_big, 0, 1)

# ---- Save images ----
save_image(orig_tensor, "original_fixed.png", normalize=False)
save_image(perturbed_highres, "cloaked_fixed.png", normalize=False)

# ---- Check predictions ----
with torch.no_grad():
    # prediction on original
    x_for_pred_orig = preprocess(orig_img).unsqueeze(0).to(device)
    probs_original = F.softmax(resnet(x_for_pred_orig), dim=1)[0]

    # prediction on perturbed
    # resize perturbed highres before model
    pert_resized = preprocess(
        transforms.ToPILImage()(perturbed_highres.squeeze(0).cpu())
    ).unsqueeze(0).to(device)
    probs_cloaked = F.softmax(resnet(pert_resized), dim=1)[0]

orig_top = torch.topk(probs_original, 5)
cloaked_top = torch.topk(probs_cloaked, 5)

print("🔍 Top classes before cloaking:")
for i in range(5):
    print(f"{idx_to_class[orig_top.indices[i]]}: {orig_top.values[i].item():.4f}")

print("\n👻 Top classes after cloaking:")
for i in range(5):
    print(f"{idx_to_class[cloaked_top.indices[i]]}: {cloaked_top.values[i].item():.4f}")


🔍 Top classes before cloaking:
tiger cat: 0.6232
Egyptian cat: 0.2579
tabby: 0.0970
radiator: 0.0034
lynx: 0.0029

👻 Top classes after cloaking:
tiger cat: 0.2612
Egyptian cat: 0.2580
tabby: 0.0952
paper towel: 0.0179
lynx: 0.0174


In [17]:
!pip install deepface

     |████████████████████████████████| 128 kB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 27.0 MB/s eta 0:00:01
  Using cached gunicorn-23.0.0-py3-none-any.whl (85 kB)
     |████████████████████████████████| 200.4 MB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 18.8 MB/s eta 0:00:01
  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
     |████████████████████████████████| 10.8 MB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 15.7 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 23.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 337 kB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 135 kB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 667 kB 15.2 MB/s eta 0:00:01
     |███████████████████████████

In [19]:
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
from PIL import Image
from facenet_pytorch import MTCNN
from torchvision import models

class VGGFace:
    @staticmethod
    def loadModel():
        # Lightweight fallback: use ImageNet-pretrained VGG16 and expose 4096-dim embeddings
        vgg = models.vgg16(pretrained=True)
        # drop final classification layer so forward() returns the penultimate 4096-d feature
        vgg.classifier = torch.nn.Sequential(*list(vgg.classifier.children())[:-1])
        return vgg
    
from torchvision.utils import save_image

device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------
# MODELS
# -------------------------
mtcnn = MTCNN(keep_all=False, device=device)

model = VGGFace.loadModel()
model.eval()
model.to(device)

# -------------------------
# TRANSFORMS
# -------------------------
face_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

to_tensor = transforms.ToTensor()


# ==========================================================
# FAWKES-STYLE FACE CLOAKING (UNTARGETED + TARGETED)
# ==========================================================
def cloak_face_fawkes(
    orig_img, 
    intensity=0.01,
    method="fgsm",
    targeted=False,
    target_identity_img=None
):
    """
    orig_img = PIL Image
    intensity = epsilon
    targeted = True/False
    target_identity_img = PIL Image (for targeted attack)
    """

    orig_w, orig_h = orig_img.size

    # ---- 1. Detect face ----
    boxes, probs = mtcnn.detect(orig_img)
    if boxes is None:
        print("No face detected")
        return orig_img

    x1, y1, x2, y2 = map(int, boxes[0])

    face_crop = orig_img.crop((x1, y1, x2, y2))

    # ---- 2. Preprocess for VGG-Face ----
    face_tensor = face_preprocess(face_crop).unsqueeze(0).to(device)
    face_tensor.requires_grad = True

    # ---- 3. Compute original embedding ----
    orig_emb = model(face_tensor).detach()

    # ---- 4. If targeted, compute target embedding ----
    if targeted:
        if target_identity_img is None:
            raise ValueError("Targeted cloaking requires target_identity_img")

        target_face_tensor = face_preprocess(target_identity_img).unsqueeze(0).to(device)
        target_emb = model(target_face_tensor).detach()
    else:
        target_emb = None

    # ---- 5. Define losses ----
    def untargeted_loss(adv_emb, orig_emb):
        # maximize distance
        return -torch.nn.functional.cosine_similarity(adv_emb, orig_emb).mean()

    def targeted_loss(adv_emb, target_emb):
        # minimize distance to target identity
        return torch.nn.functional.cosine_similarity(adv_emb, target_emb).mean()

    # ---- 6. FGSM ----
    if method == "fgsm":
        emb = model(face_tensor)

        if targeted:
            loss = targeted_loss(emb, target_emb)
        else:
            loss = untargeted_loss(emb, orig_emb)

        loss.backward()
        epsilon = intensity

        adv_small = torch.clamp(
            face_tensor + epsilon * face_tensor.grad.sign(),
            0, 1
        ).detach()

    # ---- 7. PGD ----
    elif method == "pgd":
        adv_small = face_tensor.clone()
        epsilon = intensity
        alpha = epsilon / 3
        steps = 5

        for _ in range(steps):
            adv_small.requires_grad = True
            emb = model(adv_small)

            if targeted:
                loss = targeted_loss(emb, target_emb)
            else:
                loss = untargeted_loss(emb, orig_emb)

            loss.backward()

            adv_small = adv_small + alpha * adv_small.grad.sign()
            adv_small = torch.min(torch.max(adv_small, face_tensor - epsilon), face_tensor + epsilon)
            adv_small = torch.clamp(adv_small, 0, 1).detach()

    # ---- 8. Get perturbation δ ----
    delta_small = adv_small - face_tensor

    # ---- 9. Upscale δ to original resolution ----
    face_H = y2 - y1
    face_W = x2 - x1

    delta_big = torch.nn.functional.interpolate(
        delta_small,
        size=(face_H, face_W),
        mode='bilinear',
        align_corners=False
    )

    # ---- 10. Apply only to face region ----
    orig_tensor = to_tensor(orig_img).unsqueeze(0).to(device)
    perturbed_tensor = orig_tensor.clone()

    perturbed_tensor[:, :, y1:y2, x1:x2] = torch.clamp(
        perturbed_tensor[:, :, y1:y2, x1:x2] + delta_big,
        0, 1
    )

    return perturbed_tensor


/Users/hiteshgupta/Documents/mirage-ai/ven1/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hiteshgupta/Documents/mirage-ai/ven1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
img = Image.open("input/face.png").convert("RGB")
adv = cloak_face_fawkes(img, intensity=0.01, targeted=False)
save_image(adv, "cloaked_output/face_cloaked_untargeted.png")


In [ ]:
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision.utils import save_image

device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------
# MODELS
# -------------------------
mtcnn = MTCNN(keep_all=False, device=device)

facenet = InceptionResnetV1(pretrained="vggface2").eval().to(device)

# -------------------------
# TRANSFORMS
# -------------------------
face_preprocess = transforms.Compose([
    transforms.Resize((160, 160)),   # FaceNet input size
    transforms.ToTensor()
])

to_tensor = transforms.ToTensor()


# ==========================================================
# FACE CLOAKING WITH FACENET (TARGETED + UNTARGETED)
# RETURNS METRICS
# ==========================================================
def cloak_face_facenet(
    orig_img,
    intensity=0.01,
    method="fgsm",
    targeted=False,
    target_identity_img=None
):
    orig_w, orig_h = orig_img.size

    # ---- 1. Detect face ----
    boxes, probs = mtcnn.detect(orig_img)
    if boxes is None:
        print("No face detected")
        return orig_img, {}

    x1, y1, x2, y2 = map(int, boxes[0])

    face_crop = orig_img.crop((x1, y1, x2, y2))

    # ---- 2. Preprocess ----
    face_small = face_preprocess(face_crop).unsqueeze(0).to(device)
    face_small.requires_grad = True

    # ---- 3. Original face embedding ----
    orig_emb = facenet(face_small).detach()

    # ---- 4. Optional targeted embedding ----
    if targeted:
        if target_identity_img is None:
            raise ValueError("Targeted attack requires target identity image")
        target_face = face_preprocess(target_identity_img).unsqueeze(0).to(device)
        target_emb = facenet(target_face).detach()
    else:
        target_emb = None

    # ---- 5. Loss functions ----
    def untargeted_loss(adv_emb, orig_emb):
        return -F.cosine_similarity(adv_emb, orig_emb).mean()

    def targeted_loss(adv_emb, target_emb):
        return F.cosine_similarity(adv_emb, target_emb).mean()

    # ---- 6. FGSM Attack ----
    if method == "fgsm":
        emb = facenet(face_small)
        loss = targeted_loss(emb, target_emb) if targeted else untargeted_loss(emb, orig_emb)
        loss.backward()

        epsilon = intensity
        adv_small = torch.clamp(face_small + epsilon * face_small.grad.sign(), 0, 1).detach()

    # ---- 7. PGD Attack ----
    elif method == "pgd":
        adv_small = face_small.clone()
        epsilon = intensity
        alpha = epsilon / 3
        steps = 7

        for _ in range(steps):
            adv_small = adv_small.detach()
            adv_small.requires_grad_(True)

            emb = facenet(adv_small)

            if targeted:
                loss = targeted_loss(emb, target_emb)
            else:
                loss = untargeted_loss(emb, orig_emb)

            loss.backward()

            # gradient step
            adv_small = adv_small + alpha * adv_small.grad.sign()

            # project back into epsilon-ball
            adv_small = torch.min(torch.max(adv_small, face_small - epsilon), face_small + epsilon)

            # keep valid pixel range
            adv_small = torch.clamp(adv_small, 0, 1)


    # ---- 8. Compute δ (perturbation) ----
    delta_small = adv_small - face_small

    # ---- 9. Upscale δ to original face resolution ----
    face_H = y2 - y1
    face_W = x2 - x1

    delta_big = torch.nn.functional.interpolate(
        delta_small,
        size=(face_H, face_W),
        mode='bilinear',
        align_corners=False
    )

    # ---- 10. Apply only to the face region ----
    orig_tensor = to_tensor(orig_img).unsqueeze(0).to(device)
    perturbed_tensor = orig_tensor.clone()

    perturbed_tensor[:, :, y1:y2, x1:x2] = torch.clamp(
        orig_tensor[:, :, y1:y2, x1:x2] + delta_big,
        0, 1
    )

    # ---- 11. Compute metrics ----
    adv_face_crop = transforms.ToPILImage()(perturbed_tensor[0, :, y1:y2, x1:x2].cpu())
    adv_face_small = face_preprocess(adv_face_crop).unsqueeze(0).to(device)
    adv_emb = facenet(adv_face_small).detach()

    metrics = {}

    # Untargeted metrics
    orig_sim = F.cosine_similarity(orig_emb, orig_emb).item()          # =1
    adv_sim = F.cosine_similarity(orig_emb, adv_emb).item()
    emb_dist = (orig_emb - adv_emb).norm().item()

    metrics["cosine_similarity_before"] = 1.0
    metrics["cosine_similarity_after"] = adv_sim
    metrics["embedding_distance_original_vs_adv"] = emb_dist
    metrics["similarity_drop"] = 1.0 - adv_sim

    # Targeted metrics
    if targeted:
        tgt_sim_before = F.cosine_similarity(orig_emb, target_emb).item()
        tgt_sim_after = F.cosine_similarity(adv_emb, target_emb).item()

        metrics["target_similarity_before"] = tgt_sim_before
        metrics["target_similarity_after"] = tgt_sim_after
        metrics["push_toward_target"] = tgt_sim_after - tgt_sim_before

    return perturbed_tensor, metrics


IndentationError: expected an indented block after 'elif' statement on line 84 (384942579.py, line 85)

In [30]:
img = Image.open("input/face.png").convert("RGB")
adv, metrics = cloak_face_facenet(img, intensity=0.03, targeted=False)
save_image(adv, "cloaked_output/face_cloaked_untargeted.png")
print(metrics)


{'cosine_similarity_before': 1.0, 'cosine_similarity_after': 0.6769204139709473, 'embedding_distance_original_vs_adv': 0.8038402199745178, 'similarity_drop': 0.32307958602905273}


In [31]:
my_face = Image.open("input/face.png").convert("RGB")
fake_identity = Image.open("input/celeb.jpg").convert("RGB")

adv, metrics = cloak_face_facenet(
    my_face,
    intensity=0.015,
    targeted=True,
    target_identity_img=fake_identity,
    method="pgd"
)

save_image(adv, "face_cloaked_targeted.png")
print(metrics)


RuntimeError: you can only change requires_grad flags of leaf variables.